# Introduction 👋

__`Quora Insincere Questions Classification`__ là một cuộc thi Machine Learning trên nền tảng Kaggle. Em xin phép lấy bài tập này để làm bài tổng kết cuối kỳ môn Học máy - INT3405E 20 (GV: Trần Quốc Long).

🏆 __Goal__ 
> Phân loại những câu hỏi có nội dung độc hại không phù hợp
* `input`: dữ liệu dạng `text` là câu hỏi trên `Quora` bằng tiếng anh
* `output`: `0` hoặc `1` tương đương với câu hỏi `non-toxic` hoặc `toxic` 

💡 __Idea__
> Áp dụng mô hình CNN để giải bài toán:
* CNN sử dụng các bộ lọc để trích xuất ra mối quan hệ địa phương trong những bức ảnh. Trong bài toán NLP này, ta sẽ tận dụng ưu điểm đó để xác định ngữ cảnh của câu giữa các từ.
* Để làm được điều đó, ta sẽ chuyển các câu hỏi thành các ma trận `m x n` với `n` là số chiều của mỗi từ sau khi thực hiện `word embedding`, `m` là số từ trong câu hỏi.



In [ ]:
# Libraries
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import zipfile

In [ ]:
# Make DataFrame show full screen
pd.set_option('display.max_colwidth', None)

# Root path
ROOT_PATH = "/kaggle/input/quora-insincere-questions-classification"

!ls $ROOT_PATH

# Data 📔
* `train.csv`: train_set, gồm 3 cột `qid` - id, `question_text` - nội dung câu hỏi, `target` - nhãn (0 là non-toxic, 1 là toxic)
* `test.csv`: test_set, gồm 2 cột `qid`, `question_text`, không chứa cột nhãn `target`, mình sẽ dùng để làm file `submission`
* `embeddings`: folder chứa các tập file `embeddings_word` lớn có thể dùng trong quá trình làm bài
>Ở bài tập này ta sử dụng GloVe - một dự án nguồn mở của Stanford nhằm tạo ra các vector để biểu diễn từ.

In [ ]:
TRAIN_DF = ROOT_PATH + "/train.csv"

train_df = pd.read_csv(TRAIN_DF)
train_df

In [ ]:
TEST_DF = ROOT_PATH + "/test.csv"

test_df = pd.read_csv(TEST_DF)
test_df

🎨 Ta sẽ khảo sát tính cân bằng của dữ liệu Ta sẽ khảo sát tính cân bằng của dữ liệu

In [ ]:
data_non_toxic = train_df[train_df.target == 0]
data_toxic = train_df[train_df.target == 1]

print(f"Non toxic length: {len(data_non_toxic)}")
print(f"Toxic length: {len(data_toxic)}")

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
plt.bar(x=["Non toxic", "Toxic"], height=[len(data_non_toxic), len(data_toxic)])
plt.title("Imbalanced Data", fontsize=20)
plt.xlabel("Data")
plt.ylabel("Length");

__Dữ liệu đang bị mất cân bằng__ 😰

👉 Với kỹ thuật _Undersampling_, ta sẽ tiến hành giảm số lượng dữ liệu `data_non_toxic` để tập dữ liệu được cân bằng hơn, giúp năng cao hiệu năng của mô hình
* 240,000 câu hỏi non-toxic
* 80,000 câu hỏi toxic




In [ ]:
# We get 240k non-toxic question and 80k toxic question 
train_df = pd.concat([data_non_toxic[:240000], data_toxic[:80000]])
train_df

# Text Preprocessing 🈲
👉 Như mọi khi, ta sẽ tiến hành _tiền xử lý dữ liệu_ trước khi _vector hóa dữ liệu_
* `Unicode`: chuyển văn bản về dạng unicode
* `Expand contractions`: mở rộng các từ: `i'll`, `she's`,... thành `i will`, `she is`,...
* `Lowercase`: chuyển hóa các từ thành chữ viết thường
* `Remove punctuation`: Loại bỏ dấu câu
* `Tokenize`
* `Remove stopword`: Loại bỏ các từ mà khi không có chúng thì ý nghĩa của câu vẫn không thay đổi
* `Lemmatization`: Chuyển hóa văn bản về dạng gốc nhưng vẫn giữ được ngữ cảnh của văn bản

🚫 Lưu ý

Ta dùng `Lemmatization` thay vì `Stemming` để có thể giữ được ngữ cảnh của từng từ.
>`goes` được chuyển thành `goe` khi dùng `Stemming` nhưng khi dùng `Lemmatization` thì chúng được chuyển thành `go`

Ta không loại bỏ các chữ số vì có thể chúng mang ý nghĩa lớn trong câu.
>`How did Quebec nationalists see their province as a nation in the 1960s?`  (`target`: 0)

> `Does 1 plus 1 actually equal 7? Can this claim be refuted?` (`target`: 1)

In [ ]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
from unidecode import unidecode
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def clean(text: str): 
  uni_text = str(unidecode(text).encode("ascii"), "ascii")

  expand_con = uni_text
  for word in contractions:
    expand_con = re.sub(rf"{word}", contractions[word], expand_con) 

  lower = expand_con.lower();

  remove_punc = "".join([i for i in lower if i not in string.punctuation])

  tokens = remove_punc.strip().split()

  # Keep `not` in question cause it is a word with multi meaning, that can change a non-toxic to toxic and vice versa.
  sw_list = stopwords.words()
  sw_list.remove('not')
  remove_sw = [word for word in tokens if not word in sw_list]

  # If remove stopword will remove all of `tokens`, keep at most 5 words
  if (len(remove_sw) != 0):
    lemma = [WordNetLemmatizer().lemmatize(word) for word in remove_sw]
  else:
    lemma = [WordNetLemmatizer().lemmatize(word) for word in tokens[:5]]
  return lemma

In [ ]:
# Clean data
input = []
for idx, ques in enumerate(train_df['question_text']):
  input.append(clean(ques))
  if idx % 50000 == 0:
    print(f"Completed: {idx}/320000")
print("Done")

🧶 Thêm cột dữ liệu đã xử lý vào tập dữ liệu hiện tại

In [ ]:
# Add new column after clean data into data
train_df['clean_ques_text'] = input
train_df

# Data analysis 👓

👉 Mục đích của ta khi dùng mạng CNN là biến đổi mỗi câu hỏi thành một ma trận. 

> Vì thế, ta sẽ khảo sát độ dài của các `question` để phục vụ cho việc xác định kích thước của ma trận.

In [ ]:
# Init array for questions length
tmp = [[str(idx), len(ques)] for idx, ques in enumerate(input)]
length_df = pd.DataFrame(tmp, columns=[ 'id', 'length'])
length_df.sort_values(by='length',ascending=False, inplace=True)

length_df

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(16, 7))
plt.bar(x=length_df["id"][::100], height=length_df["length"][::100])
plt.title("Questions Length", fontsize=20)
plt.xticks([])
plt.xlabel("ID")
plt.ylabel("Length");

In [ ]:
# Vocabulary count
from collections import Counter

def word_frequency(list):
  counted = Counter(list)
  word_freq = pd.DataFrame(counted.items(),columns=['word','frequency'])
  return word_freq.sort_values(by='frequency',ascending=False)

all = list()
for idx, ques in enumerate(input):
  all.extend(ques)

vocab = word_frequency(all)

In [ ]:
# Example
print(f'We have {len(all)} words')
print(f'Vocab has {len(vocab)} words')
vocab.head()

# Vector hóa dữ liệu 🎭

Trong bài tập này, ta sử dụng `GloVe` để vector hóa các từ sau khi thực hiện `clean data`.

>`GloVe` là viết tắt của `Global Vectors`, một dự án mã nguồn mở của Stanford nhằm tạo ra các vector biểu diễn cho các từ.

Sau khi `clean data`, tập từ vựng của chúng ta khá ít (~100,000 từ). Do đó, để tiết kiệm bộ nhớ cũng như tăng tốc tính toán, ta sẽ sử dụng vector 50 chiều để biểu diễn từ.
>Ta sử dụng “Wikipedia 2014 + Gigaword 5”, đây là file nhỏ nhất (“ glove.6B.zip”) có dung lượng 822 MB. Nó được huấn luyện trên một kho ngữ liệu chứa 6 tỷ từ trong tập từ vựng chứa 400,000 từ khác nhau.

🚫 Lưu ý: Khi triển khai trên `Google Colab`, em đã triển khai với `GloVe` vector 50 chiều, nhưng trong cuộc thi trên Kaggle thì chỉ có sẵn cho GloVe vector 300 chiều. Vì thế, em sẽ dùng thư viện `Word2Vec` để tự triển khai cho phù hợp với những gì đã thực hành trên `Google Colab`

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(input, vector_size=50, window=5, min_count=0, workers=4, sg=1)
model.wv.save("word.model")

In [ ]:
from gensim.models import KeyedVectors

glove = KeyedVectors.load('./word.model')

In [ ]:
# EMBEDDINGS = "../input/glove6b/glove.6B.50d.txt"

# # Init embedding words
# glove = {}
# with open(EMBEDDINGS, 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         glove[line[0]] = np.array(line[1:]).astype(np.float)

In [ ]:
# Example
print(f"Embedding words: {len(glove)}")
print(glove['convert'])

# Question to Matrix ✨
Ta sẽ tiến hành chuyển các câu hỏi thành dạng ma trận, với kích thước là số từ trong câu hỏi đó.

>Để có thể chuyển `question` thành `matrix`, ta cần xác định kích thước của `matrix` - hay chính là số từ trong `question`. 

>Dựa vào biểu đồ bên trên, ta thấy sau khi tiến hành `clean data`, các câu hỏi chủ yếu có độ dài từ 5-25.

>Câu hỏi có độ dài lớn nhất là 40. Đây là 1 con số nhỏ, vì thế, để có thể lấy được tất cả thông tin của mọi câu hỏi trong tập dữ liệu, ta sẽ khởi tạo độ lớn của ma trận `max_seq = 40`.

In [ ]:
max_seq = 40
embedding_size = 50
    
def question_embedding(question):
  matrix = np.zeros((max_seq, embedding_size))
  ques_len = len(question)

  if (ques_len > 0):
    for i in range(max_seq):
      index = i % ques_len
      if (i >= ques_len):
          break
      if(question[index] in glove):
          matrix[i] = glove[question[index]]

  if (ques_len > max_seq or ques_len == 0):
    print(question)
    print(f"Invalid: This question has {ques_len} words")

  matrix = np.array(matrix)
  return matrix

In [ ]:
# Example
print(train_df['question_text'][0])

clean_ques = train_df['clean_ques_text'][0]
print(clean_ques)

print("Embedding for last word: ")
print(question_embedding(clean_ques)[len(clean_ques)-1])
print("Embedding for last word + 1 (all is zero): ")
print(question_embedding(clean_ques)[len(clean_ques)])

# Mô tả mô hình 🎉
💡 Áp dụng mô hình CNN để giải bài toán:
* Chi tiết về [mô hình CNN](https://excessive-source-1c9.notion.site/07-10-2021-M-ng-t-ch-ch-p-CNN-aa7ebecc13a04b7bb609f2e7a9de7812#b63518c9c6d5494cbd1625c035c5ca10)
* CNN sử dụng các bộ lọc để trích xuất ra mối quan hệ địa phương trong những bức ảnh. Trong bài toán NLP này, ta sẽ tận dụng ưu điểm đó để xác định ngữ cảnh của câu giữa các từ.
* Để làm được điều đó, ta sẽ chuyển các câu hỏi thành các ma trận `m x n` với `n` là số chiều của mỗi từ sau khi thực hiện `word embedding`, `m` là số từ trong câu hỏi.

🚫 Không giống đối với đầu vào là một ảnh, do mỗi `vector` trong `matrix` biểu diễn cho 1 từ nên số chiều của `filters` trong `Convolution2D` sẽ chính bằng số chiều của `vector`.

Ta sẽ chia tập dữ liệu thành 2 tập nhỏ hơn dùng để huấn luyện mô hình:
* `train_set`: chiếm 80% - bao gồm `train_data` và `train_label`
* `valid_set`: chiếm 20% - bao gồm `valid_data` và `valid_label`

In [ ]:
data_non_toxic = train_df[train_df.target == 0]
data_toxic = train_df[train_df.target == 1]

train_set = pd.concat([data_non_toxic[:192000], data_toxic[:64000]])
valid_set = pd.concat([data_non_toxic[192000:], data_toxic[64000:]])

# Suffle DataFrame
train_set = train_set.sample(frac=1).reset_index(drop=True)
valid_set = valid_set.sample(frac=1).reset_index(drop=True)

In [ ]:
# Handle `train_set`
train_data = []
train_label = []

for ques in train_set['clean_ques_text']:
  train_data.append(question_embedding(ques))
train_data = np.array(train_data)

for y in train_set['target']:
  label = np.zeros(2)
  label[int(y)] = int(1)
  train_label.append(label)
train_label = np.array(train_label)

# Handle `valid_set`
valid_data = []
valid_label = []

for ques in valid_set['clean_ques_text']:
  valid_data.append(question_embedding(ques))
valid_data = np.array(valid_data)

for y in valid_set['target']:
  label = np.zeros(2)
  label[int(y)] = int(1)
  valid_label.append(label)
valid_label = np.array(valid_label)

len(train_data), len(train_label), len(valid_data), len(valid_label)

# Huấn luyện mô hình 🍀

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence
import datetime

sequence_length = 40
embedding_size = 50
epochs = 20
batch_size = 32
learning_rate = 0.001
decay_rate = learning_rate / epochs
dropout_rate = 0.2
num_classes = 2
filter_sizes = 3
num_filters = 25

In [ ]:
x_train = train_data.reshape(train_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_train = np.array(train_label)

x_valid = valid_data.reshape(valid_data.shape[0], sequence_length, embedding_size, 1).astype('float32')
y_valid = np.array(valid_label)

model = keras.Sequential()
model.add(layers.Convolution2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(38, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

adam = tf.optimizers.Adam(learning_rate=learning_rate, decay=decay_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())

🛫 _Training..._ 

In [ ]:
# Save the best model to .h5 file
best_model=keras.callbacks.ModelCheckpoint(filepath="best_model.h5",monitor='val_loss',save_best_only=True,verbose=1)

model.fit(x_train, y_train, batch_size=batch_size, verbose=1, epochs=epochs, validation_data=(x_valid, y_valid), callbacks=[best_model])

# Show `classification_report`
from sklearn.metrics import classification_report

y_pred = model.predict(x_valid)
y_pred = (y_pred >= 0.5) 
print("\n")
print(classification_report(y_valid, y_pred))

🪂 Free memory

In [ ]:
import gc

del data_non_toxic
del data_toxic
del train_set
del valid_set
del x_train
del y_train
del x_valid
del y_valid
del train_data
del train_label
del valid_data
del valid_label
del model
gc.collect()

# Test 🚀

In [ ]:
from keras.models import load_model
model_sentiment = load_model("best_model.h5")

In [ ]:
text = "Why do people hate Adolf Hitler? Do you have any specific and logical reasons?"
text = clean(text)

maxtrix_embedding = np.expand_dims(question_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model_sentiment.predict(maxtrix_embedding)
print("Result: ", result)
result = np.argmax(result)
print("Label predict: ", result)

# Submission 🎁

In [ ]:
# test.csv
TEST_DF = ROOT_PATH + "/test.csv"

test_df = pd.read_csv(TEST_DF)
test_df

In [ ]:
# Clean data
input = []
for idx, ques in enumerate(test_df['question_text']):
  input.append(clean(ques))
  if idx % 50000 == 0:
    print(f"Completed: {idx}/375806")
print("Done")

In [ ]:
len(input)

In [ ]:
# Add new column after clean data into data
test_df['clean_ques_text'] = input
test_df

In [ ]:
del input
gc.collect()

In [ ]:
# Prepare test data
sequence_length = 40
embedding_size = 50

test_data = []
for ques in test_df['clean_ques_text'][:200000]:
  test_data.append(question_embedding(ques))
test_data = np.array(test_data)
x_test = test_data.reshape(test_data.shape[0], sequence_length, embedding_size, 1).astype('float32')

In [ ]:
# Predict
y_test = model_sentiment.predict(x_test)
y_test = [np.argmax(idx) for idx in y_test]

TEMP_PATH = "prediction1.csv"

df_write = pd.DataFrame(y_test, columns=['prediction'])
df_write.to_csv(TEMP_PATH, index=False)

In [ ]:
del x_test
del test_data
del y_test
gc.collect()

In [ ]:
# Prepare test data
test_data = []
for ques in test_df['clean_ques_text'][200000:]:
  test_data.append(question_embedding(ques))
test_data = np.array(test_data)
x_test = test_data.reshape(test_data.shape[0], sequence_length, embedding_size, 1).astype('float32')

In [ ]:
# Predict
y_test = model_sentiment.predict(x_test)
y_test = [np.argmax(idx) for idx in y_test]

TEMP_PATH = "prediction2.csv"

df_write = pd.DataFrame(y_test, columns=['prediction'])
df_write.to_csv(TEMP_PATH, index=False)

In [ ]:
del x_test
del test_data
del y_test
del model_sentiment
gc.collect()

In [ ]:
df1 = pd.read_csv("prediction1.csv")
df2 = pd.read_csv("prediction2.csv")

df = pd.concat([df1, df2], ignore_index=True)
df

In [ ]:
# Add `prediction` column into test_dataframe
test_df["prediction"] = df["prediction"]
test_df

In [ ]:
# Export submission.csv
submission = test_df[['qid', 'prediction']]
submission.to_csv("submission.csv", index=False)
submission